In [ ]:
import os, numpy as np, pandas as pd, random, joblib
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

SEED = 2025
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
BASE = r"C:\Users\81005\Desktop\CYH\3-PFAS"
PATH_TRAIN = os.path.join(BASE, r"深度学习验证\merged_all.xlsx")
PATH_NEW   = os.path.join(BASE, r"大样本预测\7565_filled.xlsx")  
OUT_DIR    = os.path.join(BASE, r"大样本预测")
os.makedirs(OUT_DIR, exist_ok=True)

df_tr = pd.read_excel(PATH_TRAIN)
assert "Level_OA" in df_tr.columns
df_tr["Level_OA"] = df_tr["Level_OA"].astype(int)
df_new = pd.read_excel(PATH_NEW)

EXCLUDE = {"Name","Level_OA","fold_id","G","P_PI3K","P_PPAR","P_ROS","P_LPS","P_OA"}
num_cols_tr  = [c for c in df_tr.columns  if (c not in EXCLUDE and np.issubdtype(df_tr[c].dtype, np.number))]
num_cols_new = [c for c in df_new.columns if  np.issubdtype(df_new[c].dtype, np.number)]
COLS_FINAL   = [c for c in num_cols_tr if c in num_cols_new]
assert len(COLS_FINAL)>0, "训练与新数据无共同结构列"

X_all = df_tr[COLS_FINAL].replace([np.inf,-np.inf], np.nan).values
y_all = df_tr["Level_OA"].values
X_new = df_new[COLS_FINAL].replace([np.inf,-np.inf], np.nan).values

imputer = SimpleImputer(strategy="median").fit(X_all)
X_all_imp = imputer.transform(X_all)
scaler  = StandardScaler().fit(X_all_imp)
X_all_std = scaler.transform(X_all_imp)
X_new_std = scaler.transform(imputer.transform(X_new))

class TinyTabTransformer(nn.Module):
    def __init__(self, in_dim, n_classes=3, d_model=64, n_heads=4, n_layers=2, dropout=0.35):
        super().__init__()
        self.proj = nn.Linear(in_dim, d_model)
        enc = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_heads,
                                         dim_feedforward=d_model*2, dropout=dropout,
                                         batch_first=True, activation="gelu")
        self.encoder = nn.TransformerEncoder(enc, num_layers=n_layers)
        self.norm = nn.LayerNorm(d_model)
        self.head = nn.Sequential(nn.Linear(d_model, d_model//2),
                                  nn.GELU(), nn.Dropout(dropout),
                                  nn.Linear(d_model//2, n_classes))
    def forward(self, x):
        x = self.proj(x).unsqueeze(1)    
        x = self.encoder(x)
        x = self.norm(x.squeeze(1))
        return self.head(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_classes = len(np.unique(y_all))
y_all0 = (y_all - y_all.min())  

from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=SEED)
tr_idx, val_idx = next(sss.split(X_all_std, y_all0))
X_tr, X_val = X_all_std[tr_idx], X_all_std[val_idx]
y_tr, y_val = y_all0[tr_idx], y_all0[val_idx]

train_ds = TensorDataset(torch.tensor(X_tr, dtype=torch.float32),
                         torch.tensor(y_tr, dtype=torch.long))
val_ds   = TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                         torch.tensor(y_val, dtype=torch.long))
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False)

model = TinyTabTransformer(in_dim=X_all_std.shape[1], n_classes=n_classes,
                           d_model=64, n_heads=4, n_layers=2, dropout=0.35).to(device)

cls, cnt = np.unique(y_tr, return_counts=True)
w = torch.tensor([(len(y_tr)/c) for c in cnt], dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=w)
optim = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

best_f1, best_state, no_improve, patience = -1.0, None, 0, 50
from sklearn.metrics import f1_score

for ep in range(500):
    model.train()
    for bx, by in train_loader:
        bx, by = bx.to(device), by.to(device)
        logits = model(bx); loss = criterion(logits, by)
        optim.zero_grad(); loss.backward(); optim.step()
    model.eval()
    with torch.no_grad():
        pv, yv = [], []
        for bx, by in val_loader:
            bx = bx.to(device); logit = model(bx)
            pv.append(logit.argmax(1).cpu().numpy()); yv.append(by.numpy())
    f1 = f1_score(np.concatenate(yv), np.concatenate(pv), average='macro')
    if f1 > best_f1:
        best_f1, best_state, no_improve = f1, {k:v.detach().cpu().clone() for k,v in model.state_dict().items()}, 0
    else:
        no_improve += 1
        if no_improve >= patience: break

model.load_state_dict({k:v for k,v in best_state.items()})

full_ds = TensorDataset(torch.tensor(X_all_std, dtype=torch.float32),
                        torch.tensor(y_all0, dtype=torch.long))
full_loader = DataLoader(full_ds, batch_size=32, shuffle=True)
for ep in range(10):
    model.train()
    for bx, by in full_loader:
        bx, by = bx.to(device), by.to(device)
        loss = criterion(model(bx), by)
        optim.zero_grad(); loss.backward(); optim.step()

pca = PCA(n_components=5, random_state=SEED).fit(X_all_std)
T = pca.transform(X_all_std)
Pinv = np.linalg.pinv(T.T @ T)
h_train = np.einsum('ij,jk,ik->i', T, Pinv, T)
h_star  = np.quantile(h_train, 0.95)

model.eval()
with torch.no_grad():
    logits_new = model(torch.tensor(X_new_std, dtype=torch.float32).to(device)).cpu().numpy()
prob = torch.softmax(torch.tensor(logits_new), dim=1).numpy()
pred = prob.argmax(axis=1) + y_all.min()  
T_new = pca.transform(X_new_std)
h_new = np.einsum('ij,jk,ik->i', T_new, Pinv, T_new)
ad_in = (h_new <= h_star)

prob_cols = [f"Prob_Level_{i}" for i in range(1, prob.shape[1]+1)]
out = pd.concat([
    df_new.reset_index(drop=True),
    pd.DataFrame({"Pred_Level": pred, "AD_in": ad_in}),
    pd.DataFrame(prob, columns=prob_cols)
], axis=1)

out["max_prob"] = out[prob_cols].max(axis=1)
out.to_excel(os.path.join(OUT_DIR, "predictions_Xonly.xlsx"), index=False)
print("[OK] 保存：predictions_Xonly.xlsx")

top_col = prob_cols[-1]
risk_low  = out[out["Pred_Level"]==1].sort_values(top_col, ascending=False)
risk_mid  = out[out["Pred_Level"]==2].sort_values(top_col, ascending=False)
risk_high = out[out["Pred_Level"]==3].sort_values(top_col, ascending=False)

risk_low.to_excel (os.path.join(OUT_DIR, "risk_low_L1_all.xlsx"),  index=False)
risk_mid.to_excel (os.path.join(OUT_DIR, "risk_mid_L2_all.xlsx"),  index=False)
risk_high.to_excel(os.path.join(OUT_DIR, "risk_high_L3_all.xlsx"), index=False)
print("[OK] 保存：L1/L2/L3 风险清单（全体）")

core = out[(out["AD_in"]) & (out["max_prob"]>=0.80)].copy()
core[core["Pred_Level"]==1].sort_values(top_col, ascending=False)\
    .to_excel(os.path.join(OUT_DIR,"risk_low_L1_core.xlsx"), index=False)
core[core["Pred_Level"]==2].sort_values(top_col, ascending=False)\
    .to_excel(os.path.join(OUT_DIR,"risk_mid_L2_core.xlsx"), index=False)
core[core["Pred_Level"]==3].sort_values(top_col, ascending=False)\
    .to_excel(os.path.join(OUT_DIR,"risk_high_L3_core.xlsx"), index=False)
print("[OK] 保存：L1/L2/L3 风险清单（核心：AD_in & max_prob≥0.80）")

SUMMARY_TXT  = os.path.join(OUT_DIR, "AD_summary.txt")
SUMMARY_XLSX = os.path.join(OUT_DIR, "AD_breakdown.xlsx")

out["ICP_flag"] = np.select(
    [out["max_prob"]>=0.80, out["max_prob"]>=0.60],
    ["HighConf","Borderline"], default="Empty"
)
n_total          = len(out)
cov_leverage     = float(out["AD_in"].mean())
cov_highconf_all = float((out["max_prob"]>=0.80).mean())
cov_highconf_in  = float(((out["AD_in"]) & (out["max_prob"]>=0.80)).mean())

print("\n=== AD / 置信度摘要 ===")
print("样本数：", n_total)
print("Leverage h*：", float(h_star))
print("Leverage 覆盖率：", round(cov_leverage,3))
print("HighConf(≥0.80) 全体占比：", round(cov_highconf_all,3))
print("HighConf(≥0.80) AD_in内占比：", round(cov_highconf_in,3))
print("ICP_flag 分布：\n", out["ICP_flag"].value_counts(normalize=True).round(3))

with open(SUMMARY_TXT,"w",encoding="utf-8") as f:
    f.write(f"n_total={n_total}\n")
    f.write(f"h*={float(h_star):.6f}\n")
    f.write(f"cov_leverage={cov_leverage:.3f}\n")
    f.write(f"highconf_all={cov_highconf_all:.3f}\n")
    f.write(f"highconf_in_AD={cov_highconf_in:.3f}\n")
    f.write("ICP_flag:\n")
    for k,v in out["ICP_flag"].value_counts(normalize=True).items():
        f.write(f"  {k}: {v:.3f}\n")

overall_df = pd.DataFrame({
    "metric":["n_total","h_star","cov_leverage","highconf_all","highconf_in_AD"],
    "value":[n_total, float(h_star), cov_leverage, cov_highconf_all, cov_highconf_in]
})
by_icp = out["ICP_flag"].value_counts().to_frame("count")
by_icp["proportion"] = by_icp["count"]/n_total

with pd.ExcelWriter(SUMMARY_XLSX) as w:
    overall_df.to_excel(w, sheet_name="overall", index=False)
    by_icp.to_excel(w, sheet_name="by_ICP_flag")
    out[["AD_in","ICP_flag","max_prob",top_col]+prob_cols].head(20)\
        .to_excel(w, sheet_name="preview_top20", index=False)

print("[OK] 保存：AD_summary.txt / AD_breakdown.xlsx")
